In [1]:
from collections import defaultdict
from collections import OrderedDict 
import pandas as pd

In [2]:
wdf = pd.read_csv('warehouse_to_society_dist_sample.csv')
sdf = pd.read_csv('society_to_society_dist_sample.csv')

maxOrderCount = 350   # max amount of goods that can be delivered via 1 truck
maxTime = 135*60        # 135 minutes converted to seconds

# taking few rows
wdf = wdf.head(26)

In [3]:
# Converting the delivery time to seconds
def time_convert(x):
    h,m,s = map(int,x.split(':'))
    return (h*60+m)*60+s

wdf['Delivery_Time'] = wdf['Delivery Time'].apply(time_convert)

In [4]:
wdf.drop('latitude', axis=1, inplace=True)
wdf.drop('longitude', axis=1, inplace=True)
wdf.drop('warehouse_dist', axis=1, inplace=True)    # not using these in algo, makes more readable

In [5]:
wdf.head()

,Society ID,Order Value,Order Count,Delivery Time,warehouse_time,Delivery_Time
0,1,"94,288",605,1:08:42,1722,4122
1,2,"29,180",168,1:15:57,1409,4557
2,3,"80,801",431,1:57:06,1331,7026
3,7,"38,084",216,1:18:04,1466,4684
4,8,"28,952",212,1:05:29,1202,3929


In [6]:
sdf.drop('latitude_x', axis=1, inplace =True)
sdf.drop('latitude_y', axis=1, inplace =True)
sdf.drop('longitude_x', axis=1, inplace =True)
sdf.drop('longitude_y', axis=1, inplace =True)
sdf.drop('Distance', axis=1, inplace =True)                   # not using these either
sdf.head()

,society_1,society_2,Time
0,1,1,0
1,2,1,914
2,3,1,898
3,7,1,1000
4,8,1,1024


In [7]:
print(wdf.shape)
print(sdf.shape)

(26, 6)
(676, 3)


In [8]:
wdf.head()

,Society ID,Order Value,Order Count,Delivery Time,warehouse_time,Delivery_Time
0,1,"94,288",605,1:08:42,1722,4122
1,2,"29,180",168,1:15:57,1409,4557
2,3,"80,801",431,1:57:06,1331,7026
3,7,"38,084",216,1:18:04,1466,4684
4,8,"28,952",212,1:05:29,1202,3929


In [9]:
wdf['total_time'] = wdf['warehouse_time'] + wdf['Delivery_Time']
wdf.drop('Delivery Time', axis=1, inplace=True)
wdf.sort_values('total_time', inplace = True)
wdf.head()

,Society ID,Order Value,Order Count,warehouse_time,Delivery_Time,total_time
22,27,"8,431",53,1115,769,1884
20,25,"4,211",29,1346,919,2265
19,24,"5,957",24,1282,1092,2374
8,12,"10,146",54,1380,1352,2732
5,9,"11,869",65,1050,2341,3391


In [10]:
orderCount        = defaultdict( lambda: 0 )          # stored the order count, if exceeding 350 then find remainder 
fullyLoadedTrucks = defaultdict( lambda: 0 )          # will store how many fully loaded trucks shall be sent to society with 350 orders
for index, row in wdf.iterrows():
    orderCount[row['Society ID']] = (int(row['Order Count']))

for i in orderCount:                                  # implementation of above mentioned comments.
    if orderCount[i] > maxOrderCount:
        fullyLoadedTrucks[i] = orderCount[i]// maxOrderCount      
        # quotient = no. of fully loaded truck for eg. OrderCount for some society id = 1225, then we need to send 
        # 3 fullyloaded trucks and one partially filled with 175 orders
        # 1225//350 = 3, 1225 % 350 = 175
        orderCount[i] %= maxOrderCount
        

In [11]:
# add ordercount and number of fully loaded trucks required in dataframe
wdf['Fully Loaded Trucks'] = wdf['Society ID']
wdf['Fully Loaded Trucks'] = wdf['Fully Loaded Trucks'].map(fullyLoadedTrucks)
wdf['Order Count'] = wdf['Society ID']
wdf['Order Count'] = wdf['Order Count'].map(orderCount)
wdf.head()

,Society ID,Order Value,Order Count,warehouse_time,Delivery_Time,total_time,Fully Loaded Trucks
22,27,"8,431",53,1115,769,1884,0
20,25,"4,211",29,1346,919,2265,0
19,24,"5,957",24,1282,1092,2374,0
8,12,"10,146",54,1380,1352,2732,0
5,9,"11,869",65,1050,2341,3391,0


In [12]:
timeFromWH   = OrderedDict()   # time spent from going from wh to society
deliveryTime = dict()          # time spent during delivery within the society


for _, row in wdf.iterrows():
    sid = int(row['Society ID'])
    timeFromWH[sid] = int(row['total_time'])
    
    deliveryTime[sid] = int(row['Delivery_Time'])

In [13]:
for _, row in sdf.iterrows():
    sid = int(row['society_1'])
    sdf.at[_, 'Time'] =  deliveryTime[sid] + int(row['Time'])
    # set 'time' of i'th society equal to time spent in delivering goods in that society + time spent on traveling

sdf.head()

,society_1,society_2,Time
0,1,1,4122
1,2,1,5471
2,3,1,7924
3,7,1,5684
4,8,1,4953


In [14]:
sdf.sort_values(['society_2','Time'], inplace= True)        # closest society first.
sdf.head()

,society_1,society_2,Time
20,25,1,1495
22,27,1,1691
19,24,1,1745
8,12,1,2174
24,29,1,3087


In [15]:
time = defaultdict(OrderedDict)
for _, row in sdf.iterrows():
    _from = int(row['society_2'])
    _to = int(row['society_1'])
    if _from == _to: continue
    time[_from][_to] = int(row['Time'])                     
    # MOST IMPORTANT: if we decide to go from society x to y then how much time will we require on travel and delivering goods in society y,
    # NOTE: we don't consider the time spent in societey x in time[x][y], we assume we have already somehow reached x
    # this loop is the reason why overall time complexity of this code reaches O(n**2)


In [16]:
visited = defaultdict(lambda : False) # No society is visited
CountOfVisited = 0 # initially 0 societies are visited
path = defaultdict(lambda : [])
truck = 1 # This variable will give the total number of trucks required
truckNumber = defaultdict(lambda: []) # This dict will store the trucknumber of i'th society
truckOrder = defaultdict(lambda: [])
considered = defaultdict(lambda: True)

while CountOfVisited != len(wdf):
    # until all societies are visited
    if not len(timeFromWH): break
    society, timereq = timeFromWH.popitem(last=False)     
    '''  ^^^^^^
    pops out closest society and timereq for both: going from warehouse to the society and delivering goods in the society 
    
    '''
    while visited[society]:
        society, timereq = timeFromWH.popitem(last=False)    # closest society has already been visited then find next closest
    
    if timereq > maxTime or orderCount[society]>maxOrderCount:           
        # if total time required is more than MaxTime, then we don't consider going to this society
        considered[society] = False
        visited[society] = True
        truckNumber[society] = -1  # not considering
        truckOrder[society ] = -1  # not considering
        CountOfVisited += 1
        continue
    
    ordersf = orderCount[society]
    timesf = timereq               # we went to closest society and time_so_far = delivery time + travel time
    queue = 1                      # this society is where the truck shall go first (truck Order)
    while timesf < maxTime and ordersf < maxOrderCount: 
        # if we have more time left then continue finding the closest society and deliver goods.
        visited[society] = True    # mark as current society visited
        CountOfVisited  += 1
        truckNumber[society] = truck    
        truckOrder[society]  = queue
        if CountOfVisited == len(wdf): break
        queue += 1    
        society, timreq = time[society].popitem(last=False)
        while visited[society]:
            society, timereq = time[society].popitem(last=False) #find next closest and repeat.
        timesf += timereq
        ordersf += orderCount[society]
    
    # if we are here, then truck went back from societies due to time limit, and we need to send other trucks there to
    # deliver goods and hence increment the truck number by one and repeat this whole process
    truck += 1              
    

In [17]:
# adding info to dataframe
wdf['Truck Number'] = wdf['Society ID']
wdf['Truck Number'] = wdf['Truck Number'].map(truckNumber)
wdf['Truck Order']  = wdf['Society ID']
wdf['Truck Order']  = wdf['Truck Order'].map(truckOrder)
#wdf.sort_values(['Truck Number', 'Truck Order'], inplace= True)
wdf.head(len(wdf))

,Society ID,Order Value,Order Count,warehouse_time,Delivery_Time,total_time,Fully Loaded Trucks,Truck Number,Truck Order
22,27,"8,431",53,1115,769,1884,0,1,1
20,25,"4,211",29,1346,919,2265,0,1,2
19,24,"5,957",24,1282,1092,2374,0,1,3
8,12,"10,146",54,1380,1352,2732,0,1,4
5,9,"11,869",65,1050,2341,3391,0,2,1
21,26,"14,467",109,1250,2213,3463,0,3,1
24,29,"29,364",156,1384,2428,3812,0,2,2
6,10,"10,803",59,1355,2580,3935,0,4,1
12,17,"35,536",151,2341,1912,4253,0,3,2
14,19,"21,893",118,1682,2605,4287,0,4,2


In [18]:
wdf.to_csv('output.csv', index=False)